# Assignment 3

## Questions

    1. What is the purpose of interleavers?
    
   Interleavers are used in order to decrease the BER of a transmitted message. It tries to distribute a codeword in a large time so that the effect of fading in the channel gets minimized.
    
    2. Why does it _not_ improve the mean BER for uncoded system?
    
   Because without coding there is no codeword and redundancy to try to find the original codeword. There is no improvement in changing the order of the bits in an uncoded system, the channel will behave the same and the probability of getting one bit wrong is the same. 
    
    3. What is the latency for the block interleaver in Figure 14.26?
    
   The latency will be $N_{inteleave}D_{interleave}T_B$ since a codeword will take this time to fill the block and be sent.
    
    4. How long is the distance between two adjacent bits after interleaving?
    
   Taking figure 14.26 as the example, the first two bits that are represented in the first column will be read after all the bits in the first row be read completely. Therefore, the distance will become $N_{interleave}T_B$, that is the number of columns in the block times the bit duration.
    


## Observation about the functions that perform the interleaving

As I am the one who is going to design the system, I can choose the format of the input of this function. I find it easier to see the interleave as a simple transposition of a matrix. The dimensions of the matrix can be chosen as 7x7, this means that it will do the interleaving procedure in blocks of 7 codewords of length 7. That option is justified by the previous use of the Hamming(4,7) coding.

## Interleave and deinterleave procedures

With the help of my code for assignment 2 I built the input matrix $A$ composed of valid codewords generated by the generator matrix used in that assignment. This can be done by just taking any 4 digit binary number as the input of the function encode and getting the output.

In [34]:
import numpy as np

def interleave(A):
    return np.transpose(A)

def deinterleave(A):
    return np.transpose(A)

A=np.matrix('1 ,0 ,0 ,0 ,1 ,1 ,0;1 ,1 ,0 ,0 ,0 ,1 ,1;1 ,1 ,1 ,0 ,0 ,0 ,0;1 ,1 ,1 ,1 ,1 ,1 ,1;0, 1, 1, 1, 0, 0, 1;0, 0, 1, 1, 1, 0, 0;0, 0, 0, 1, 1, 1, 1')
print(A) #This is the original matrix, being printed for comparision later

[[1 0 0 0 1 1 0]
 [1 1 0 0 0 1 1]
 [1 1 1 0 0 0 0]
 [1 1 1 1 1 1 1]
 [0 1 1 1 0 0 1]
 [0 0 1 1 1 0 0]
 [0 0 0 1 1 1 1]]


Suppose that we read the codewords from the first column to the last one, where the first raw of the matrix corresponds to the first codeword.
After interleaving, the transmit matrix is (compare with the first print)

In [35]:
print(interleave(A))

[[1 1 1 1 0 0 0]
 [0 1 1 1 1 0 0]
 [0 0 1 1 1 1 0]
 [0 0 0 1 1 1 1]
 [1 0 0 1 0 1 1]
 [1 1 0 1 0 0 1]
 [0 1 0 1 1 0 1]]


So, just to be clear, in this case the first codeword to be transmited would be $[1 0 0 0 0 0 0]$.

Now, after doing the interleaving, we transmit the signal and get a fading dip at some values of the matrix, for instance on the components $a_{4,4}$, $a_{4,5}$ and $a_{4,6}$. It is important to state that, in this case, the transmission of the bits are in the same order as they are read, that is why the fading dip will cause problems in the neighboring components in a row. Let us say that unfortunately at this components the received values are exactly the wrong digits, then the receiving matrix will be

$$B=\begin{bmatrix}
 1&1&1&1&0&0&0\\
 0&1&1&1&1&0&0\\
 0&0&1&1&1&1&0\\
 0&0&0&0&0&0&1\\
 1&0&0&1&0&1&1\\
 1&1&0&1&0&0&1\\
 0&1&0&1&1&0&1\\
\end{bmatrix}$$

If the function deinterleave is used in this matrix we get

In [36]:
B=interleave(A)
B[3,3]=0;
B[3,4]=0;
B[3,5]=0;
A_hat=deinterleave(B)
print(A_hat)

[[1 0 0 0 1 1 0]
 [1 1 0 0 0 1 1]
 [1 1 1 0 0 0 0]
 [1 1 1 0 1 1 1]
 [0 1 1 0 0 0 1]
 [0 0 1 0 1 0 0]
 [0 0 0 1 1 1 1]]


We observe that the 4th, 5th and the 6th codewords of this matrix have a bit error. That is not a problem since we can correct that in the decoding process. For instance, by taking the 4th codeword of the received matrix as the input of the function decode used in the last assignment we get

In [37]:
def decode(x_hat):
    H = np.matrix('1,1,0,1,1,0,0;1,0,1,1,0,1,0;0,1,1,1,0,0,1') # eq. 14.33 of the  textbook
    s=x_hat*H.transpose() # Defining the syndrome vector
    s=s%2 # Taking in binary digits
    
    # Now we check which column of the transpose of matrix H the syndrome vector correspond to and correct the input 
    # vector in that position
    
    if s[0,0]==0 and s[0,1]==0 and s[0,2]==0:
        return [x_hat[0],x_hat[1],x_hat[2],x_hat[3]]
    else:
        for i in range(7):
            if s[0,0]==H.transpose()[i,0] and s[0,1]==H.transpose()[i,1] and s[0,2]==H.transpose()[i,2]:
                break
        x_hat[i]=x_hat[i]+1
        x_hat[:]=[y%2 for y in x_hat]
        return [x_hat[0],x_hat[1],x_hat[2],x_hat[3]]
    
print(decode([1, 1, 1, 0, 1, 1, 1]))

[1, 1, 1, 1]


We can verify the validity of the decoding by taking the actual uncoded codeword transmited from the original matrix A $[1 1 1 1 1 1 1]$ and decode it

In [38]:
print(decode([1,1,1,1,1,1,1]))

[1, 1, 1, 1]


The result checks!